# Association of chrM2158T>C variant with PD risk in AllOfUs
* **Project:** Mitochondrial 2158T>C variant in PD
* **Version:** Python/3.9
* **Status:** COMPLETE
* **Last Updated:** 07-APRIL-2024

### Notebook Overview
Querying AllOfUs to extract maternal proxy cases CRAM files, merge them, use Mutect2 and PLINK2 for analysis

In [3]:
import pandas
import os

# This query represents dataset "pd_maternal_proxy" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_46445045_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (836812) 
                                AND is_standard = 0  
                                AND  value_source_concept_id IN (43529690)
                            )) criteria ) 
                        AND cb_search_person.person_id IN (SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            has_whole_genome_variant = 1 ) 
                        AND cb_search_person.person_id NOT IN (SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    concept_id IN (381270) 
                                                    AND full_text LIKE '%_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                is_standard = 1 
                                                AND is_selectable = 1
                                            ) 
                                            AND is_standard = 1 
                                    )
                                ) criteria 
                            ) )"""

dataset_46445045_person_df = pandas.read_gbq(
    dataset_46445045_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")


# To write the DataFrame to a CSV file:
dataset_46445045_person_df.to_csv("mar2_matproxy_person_df.csv", index=False)

Downloading:   0%|          | 0/1110 [00:00<?, ?rows/s]

In [2]:
import pandas
import os

# This query represents dataset "pd_maternal_proxy" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_46445045_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                836812
            ) 
            OR question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1586134
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (836812) 
                                        AND is_standard = 0  
                                        AND  value_source_concept_id IN (43529690)
                                    )) criteria ) 
                                AND cb_search_person.person_id IN (SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_whole_genome_variant = 1 ) 
                                AND cb_search_person.person_id NOT IN (SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (381270) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) ))"""

dataset_46445045_survey_df = pandas.read_gbq(
    dataset_46445045_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_46445045_survey_df.head(5)
# To write the DataFrame to a CSV file:
dataset_46445045_survey_df.to_csv("mar2_matproxy_survey_df.csv", index=False)

Downloading:   0%|          | 0/24488 [00:00<?, ?rows/s]

In [5]:
#Include only European samples
ancestry_pred_path = "gs://${ALLOFUS}/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv"
os.environ['ancestry_pred_path'] = ancestry_pred_path
!gsutil -u $GOOGLE_PROJECT cat $ancestry_pred_path | cut -f1,2 | grep eur > eur_ancestry
!grep -v person mar2_matproxy_person_df.csv | cut -f1 -d "," > list_of_all_mat_proxy
!for i in `cat list_of_all_mat_proxy`; do grep $i eur_ancestry | cut -f1 >> eur_mat_cases ; done 

In [14]:
#Exclude flagged and related samples
!grep -v -F -f flagged_samples eur_mat_cases > eur_mat_unre_cases
!grep -v -F -f flagged_samples eur_mat_cases | wc -l

902


In [ ]:
#CRAM streaming for chrM

In [2]:
## Package Import
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_ID={LINE_COUNT_JOB_ID}
## Defining necessary pathways
my_bucket = os.environ['WORKSPACE_BUCKET']
my_bucket

In [15]:
!head -1 manifest.csv > manifest_maternal.csv

In [16]:
!for i in `cat eur_mat_unre_cases` ; do grep $i manifest.csv >> manifest_maternal.csv ; done

In [16]:
!wc -l manifest_maternal.csv

In [15]:
!mkdir CRAM_Streaming_mat
!cp manifest_maternal.csv CRAM_Streaming_mat/.
cram_manifest = pd.read_csv('CRAM_Streaming_mat/manifest_maternal.csv')
cram_manifest['cram_uri'].iloc[0]

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env USER_NAME={USER_NAME}
## MODIFY FOR FULL DATA RUN
# Use hyphens, not whitespace since it will become part of the bucket path.
JOB_NAME='CRAM_Stream_Test_mat'

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_NAME={JOB_NAME}

In [ ]:
## Analysis Results Folder
line_count_results_folder = os.path.join(
    os.getenv('WORKSPACE_BUCKET'),
    'dsub',
    'results',
    JOB_NAME,
    USER_NAME,
    datetime.now().strftime('%Y%m%d'))

line_count_results_folder

In [ ]:
## Where the output files will go
output_files = os.path.join(line_count_results_folder, "results")
print(output_files)

In [ ]:
OUTPUT_FILES = output_files


# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env OUTPUT_FILES={OUTPUT_FILES}

In [23]:
cram_count = len(cram_manifest)
cram_count
jobs = cram_count/50
jobs

18.04

In [17]:
##Creating batches of 10 will be larger for samples
##Crams 2-101
##For full just making a df with all crams
test_crams = cram_manifest['cram_uri'].iloc[0:]

In [25]:
## Made a batch folder
!mkdir CRAM_Streaming_mat/batches
!realpath CRAM_Streaming_mat/

/home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_mat


In [34]:
## Splitting into ten files with ten cram pathways per 
## Copy batch realpath output above into '' in the to_csv command being sure to keep the single quotes
## At the end of the path, be sure to add /cram_batch_{id}.txt 
for id, test_crams_i in  enumerate(np.array_split(test_crams, 30)):
    test_crams_i.to_csv('/home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_mat/batches/cram_batch_{id}.txt'.format(id=id), index=False, header=None)

In [18]:
## Coping batches to user directory
!gsutil -m cp CRAM_Streaming_mat/batches/* {my_bucket}/dsub/final_batches/

In [36]:
!ls CRAM_Streaming_mat/batches/ | wc -l
!wc -l CRAM_Streaming_mat/batches/cram_batch_8.txt

30
30 CRAM_Streaming_mat/batches/cram_batch_8.txt


In [38]:
##Generated batch file
!gsutil ls {my_bucket}/dsub/final_batches/*.txt > AoU_batches_mat.txt

In [40]:
## Move batch file
## Use realpath output above again in this command:
!mv AoU_batches_mat.txt /home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_mat/

In [41]:
!realpath CRAM_Streaming_mat/AoU_batches_mat.txt

/home/jupyter/workspaces/neurologicalgenescreening/CRAM_Streaming_mat/AoU_batches_mat.txt


In [42]:
%%writefile ~/printreads_depth_CRAM_Stream.sh

set -o pipefail
set -o errexit

# ---------Required Inputs---------
# aou_crams - A .txt file containing gs:// paths to cram samples.

# Given a .txt file - get X samples.
# For parallel submissions:
# - Use a different .txt file per submission.
# - Each .txt file can contain a different number of lines
aou_crams_len=$(wc -l < ${aou_crams})
echo "Samples in cramlist: ${aou_crams_len}"

# ---------Required Output---------
#filtered_cram_output

echo "GOOGLE_PROJECT: ${GOOGLE_PROJECT}"
echo "OUTPUT_PATH: ${OUTPUT_PATH}"
echo "ref_dict: ${ref_dict}"
echo "ref_fai: ${ref_fai}"
echo "ref_fasta: ${ref_fasta}"

# Perform runs for x samples.
for (( i=1; i<$aou_crams_len+1; i++ ));
do

    # These change per iteration
    export aou_cram_reads=$(sed "${i}!d;q" "${aou_crams}")   # gs:// path to a cram sample
    export aou_cram_reads_name=`basename ${aou_cram_reads}`  # file_name.cram
    export aou_cram_reads_prefix="${aou_cram_reads_name%.*}" # file_name
    echo "aou_cram_reads: ${aou_cram_reads}"
    echo "aou_cram_reads_name: ${aou_cram_reads_name}"
    echo "aou_cram_reads_prefix: ${aou_cram_reads_prefix}"

    # ----------------------------------WORKFLOW----------------------------------

    # Stream CRAM found at gs:// path into a new, smaller CRAM based on genomic intervals given with -L.
    /gatk/gatk PrintReads \
        -I ${aou_cram_reads} \
        -L "chrM" \
        -R "${ref_fasta}" \
        -O "${aou_cram_reads_prefix}_mt_control.cram" \
        --gcs-project-for-requester-pays ${GOOGLE_PROJECT} \
        --cloud-prefetch-buffer 0 --cloud-index-prefetch-buffer 0

    # Outputs
    export igk_depth_cram="${aou_cram_reads_prefix}_mt_control.cram"
    echo "igk_depth_cram: ${igk_depth_cram}"

    # Disk space
    echo "Disk space taken up so far:"
    du -d 1 -h
    echo "${i} run(s) finished."

    # Move results to output directory
    mv ${igk_depth_cram} ${OUTPUT_PATH}
done

Overwriting /home/jupyter/printreads_depth_CRAM_Stream.sh


In [43]:
!gsutil cp /home/jupyter/printreads_depth_CRAM_Stream.sh {my_bucket}/dsub/scripts/

Copying file:///home/jupyter/printreads_depth_CRAM_Stream.sh [Content-Type=text/x-sh]...
/ [1 files][  1.9 KiB/  1.9 KiB]                                                
Operation completed over 1 objects/1.9 KiB.                                      


In [ ]:
# Copy this cell's output to the BASH_SCRIPT variable in cell 33, the dsub command below
!gsutil ls {my_bucket}/dsub/scripts/

In [45]:
!cp CRAM_Streaming_mat/AoU_batches_mat.txt .

In [ ]:
%%bash --out LINE_COUNT_JOB_ID

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

# Get all cramlists
bashArray=()

## ------------------------------------------------ MAKE CHANGES HERE ------------------------------------------
#Change the 'done < test_cram_batch.txt' to 'done < AoU_v7_batches.txt' if you want to run across all batches
while read line; do
  bashArray+=($line)
done < AoU_batches_mat.txt
## -------------------------------------------------------------------------------------------------------------

# Length of entire array
len_bashArray=${#bashArray[@]}

LOWER=0
UPPER=$len_bashArray
MACHINE_TYPE="n2-standard-4"
## ------------------------------------------------ MAKE CHANGES HERE ------------------------------------------
DATE=20230301
BASH_SCRIPT="${WORKSPACE_BUCKET}/dsub/scripts/printreads_depth_CRAM_Stream.sh"
## -------------------------------------------------------------------------------------------------------------

for ((batch=$LOWER;batch<$UPPER;batch+=1))
do
    dsub \
        --provider google-cls-v2 \
        --user-project "${GOOGLE_PROJECT}"\
        --project "${GOOGLE_PROJECT}"\
        --network "${AOU_NETWORK}" \
        --subnetwork "${AOU_SUBNETWORK}" \
        --service-account "$(gcloud config get-value account)" \
        --user "${DSUB_USER_NAME}" \
        --regions us-central1 \
        --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/${DATE}/${bashArray[batch]}.log" \
        "$@" \
        --preemptible \
        --boot-disk-size 100 \
        --machine-type ${MACHINE_TYPE} \
        --disk-size 100 \
        --name "${JOB_NAME}" \
        --script "${BASH_SCRIPT}" \
        --image 'gcr.io/bick-aps2/briansha/pileup_preprocessing:latest' \
        --env GOOGLE_PROJECT=${GOOGLE_PROJECT} \
        --input ref_dict="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.dict" \
        --input ref_fai="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.fasta.fai" \
        --input ref_fasta="${WORKSPACE_BUCKET}/data/Homo_sapiens_assembly38.fasta" \
        --input aou_crams="${bashArray[batch]}" \
        --output-recursive OUTPUT_PATH="${OUTPUT_FILES}/${batch}"
done

In [48]:
!dstat --provider google-cls-v2 --project ${GOOGLE_PROJECT} --location us-central1 --jobs 'cram-strea--fulyaakcimen--240308-220245-41' --users 'fulyaakcimen' --status '*'

Job Name         Status    Last Update
---------------  --------  --------------
cram-stream-...  Success   03-08 22:08:47



In [14]:
!for i in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29  ; do  gsutil -o 'GSUtil:parallel_thread_count=20' -m cp -r ${WORKSPACE_BUCKET}/dsub/results/CRAM_Stream_Test_mat/fulyaakcimen/20240308/results/$i/wgs_*_mt_control.cram mat_prox/. ; done

In [ ]:
#index cram files

In [8]:
!for i in `cat proxy_crams` ; do samtools index $i ; done

In [9]:
!head -100 proxy_crams > proxy_crams_1
!head -200 proxy_crams | tail -100 > proxy_crams_2
!head -300 proxy_crams | tail -100 > proxy_crams_3
!head -400 proxy_crams | tail -100 > proxy_crams_4
!head -500 proxy_crams | tail -100 > proxy_crams_5
!head -600 proxy_crams | tail -100 > proxy_crams_6
!head -700 proxy_crams | tail -100 > proxy_crams_7
!head -800 proxy_crams | tail -100 > proxy_crams_8
!head -902 proxy_crams | tail -102 > proxy_crams_9

In [6]:
#run the for loop for 9 proxy_crams batches
!for i in `cat proxy_crams_1` ; do gatk --java-options "-Xmx10G" Mutect2 -R Homo_sapiens_assembly38.fasta -L chrM --mitochondria-mode -I $i -O "$i".vcf.gz ; done

In [42]:
!bcftools merge mat_prox/*gz -Oz -o merged_mat.vcf.gz --force-samples -r chrM:2157-2159

In [8]:
!plink2 --vcf merged_mat.vcf.gz --make-bed --double-id --out merged_mat_eur \
--chr MT --from-bp 2158 --to-bp 2158

In [9]:
!plink --bfile merged_mat_eur --fill-missing-a2 --make-bed --out merged_mat_eur_misupd

In [10]:
!plink2 --bfile merged_mat_eur_misupd --freq --out mat_freq

In [11]:
!plink --bfile  merged_mat_eur_misupd --bmerge PDfiltecontrols --make-bed --out PD_matcontrols

In [13]:
!plink2 --bfile PD_matcontrols --glm firth-fallback --pheno mat_pheno --pheno-name PHENO --ci 0.95 --vif 100 \
--out mat_proxytest --covar mat_pheno --covar-name age,SEX,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10  --covar-variance-standardize

In [1]:
!cat mat_proxytest.PHENO.glm.logistic.hybrid

#CHROM	POS	ID	REF	ALT	A1	FIRTH?	TEST	OBS_CT	OR	LOG(OR)_SE	L95	U95	Z_STAT	P
MT	2158	.	T	C	C	N	ADD	13544	0.685152	0.806072	0.141142	3.32596	-0.469082	0.639011
MT	2158	.	T	C	C	N	PC1	13544	0.972167	0.0560921	0.870954	1.08514	-0.503242	0.614794
MT	2158	.	T	C	C	N	PC2	13544	1.15029	0.0740261	0.994935	1.32989	1.89137	0.0585745
MT	2158	.	T	C	C	N	PC3	13544	0.906594	0.0701327	0.790162	1.04018	-1.39821	0.16205
MT	2158	.	T	C	C	N	PC4	13544	0.878755	0.100063	0.722262	1.06916	-1.29168	0.196469
MT	2158	.	T	C	C	N	PC5	13544	0.970547	0.0735408	0.840269	1.12102	-0.406521	0.68436
MT	2158	.	T	C	C	N	PC6	13544	1.15911	0.467093	0.464016	2.89544	0.316103	0.751924
MT	2158	.	T	C	C	N	PC7	13544	0.870048	0.490016	0.332997	2.27324	-0.284087	0.776344
MT	2158	.	T	C	C	N	PC8	13544	0.92398	0.168233	0.664451	1.28488	-0.469973	0.638374
MT	2158	.	T	C	C	N	PC9	13544	0.928382	0.0644666	0.818187	1.05342	-1.15272	0.249026
MT	2158	.	T	C	C	N	PC10	13544	1.04365	0.0417646	0.961627	1.13268	1.02307	0.306274
MT	2158	.	T	C	C	N	SEX=Female	